<a href="https://colab.research.google.com/github/Kotaro015/dl_lecture_competition_pub/blob/main/DL_Basic_2025_Competition_NYUv2_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning 基礎講座　最終課題: NYUv2 セマンティックセグメンテーション

## 概要
RGB画像から、画像内の各ピクセルがどのクラスに属するかを予測するセマンティックセグメンテーションタスク.

### データセット
- データセット: NYUv2 dataset
- 訓練データ: 795枚
- テストデータ: 654枚
- 入力: RGB画像 + 深度マップ（元画像サイズは可変）
- 出力: 13クラスのセグメンテーションマップ
- 評価指標: Mean IoU (Intersection over Union)

### データセットの詳細（[NYU Depth Dataset V2](https://cs.nyu.edu/~fergus/datasets/nyu_depth_v2.html)）
- 画像は屋内シーンを撮影したもので、家具や壁、床などの物体が含まれています.
- 各画像に対して13クラスのセグメンテーションラベルが提供されます.
- データは以下のディレクトリ構造で提供:
```
data/NYUv2/
├─train/
│  ├─image/      # RGB画像
│  │    000000.png
│  │    ...
│  │
│  ├─depth/      # 深度マップ
│  │    000000.png
│  │    ...
│  │
│  └─label/      # 13クラスセグメンテーション（教師ラベル）
│       000000.png
│       ...
└─test/
   ├─image/      # RGB画像
   │    000000.png
   │    ...
   │  ├─depth/   # 深度マップ
   │    000000.png
   │    ...
```

### タスクの詳細
- 入力のRGB画像と深度マップから、各ピクセルが13クラスのどれに属するかを予測するタスクです.
- 評価はMean IoUを使用します．
  - 各クラスごとにIoUを計算し、その平均を取ります.
  - IoUは以下の式で計算:
  $$IoU = \frac{TP}{TP + FP + FN}$$
    - TP: True Positive（正しく予測されたピクセル数）
    - FP: False Positive（誤って予測されたピクセル数）
    - FN: False Negative（見逃したピクセル数）

### 前処理
- 入力画像は512×512にリサイズされます.
- ピクセル値は0-1に正規化されます.
- セグメンテーションラベルは0-12の整数値（13クラス）です．
  - 255はignore index（評価から除外）

### 提出形式
- テスト画像（RGB + Depth）の各ピクセルに対してクラス（0~12）を予測したものをnumpy配列として保存されます.
- ファイル名: `submission.npy`
- 配列の形状: [テストデータ数, 高さ, 幅]
- 各ピクセルの値: 0-12の整数（予測クラス）



## 考えられる工夫の例
- 事前学習モデルの fine-tuning
    - ImageNetなどで事前学習されたモデルを本データセットでfine-tuningすることで性能向上が見込めます.
- 損失関数の再設計
    - クラスごとの出現頻度に応じて損失を補正するように損失関数を設計すると、クラス分布の不均衡に対してロバストな学習ができます.
- 画像の前処理
    - RandomResizedCrop / Flip / ColorJitter 等のデータ拡張を追加することで，汎化性能の向上が見込めます．

## 修了要件を満たす条件
- ベースラインでは，omnicampus 上での性能評価において， 38.2% となります．したがって，ベースラインである 38.2% を超えた提出のみ，修了要件として認めます．
- ベースラインから改善を加えることで， 50%以上に性能向上することを運営で確認しています．こちらを 1つの指標として取り組んでみてください．

## 注意点
- 学習するモデルについて制限はありませんが，必ず訓練データで学習したモデルで予測してください．
    - 事前学習済みモデルを利用して，訓練データを fine-tuning しても構いません．
    - 埋め込み抽出モデルなど，モデルの一部を訓練しないケースは構いません．
    - 学習を一切せずに，ChatGPT などの基盤モデルを利用することは禁止とします．

### データの準備
データをダウンロードした際に，google drive したため，利用するために google drive をマウントする必要があります．また， drive 上で展開することができないため，/content ディレクトリ下にコピーし "data.zip" を展開します．  
google drive 上に "data.zip" が配置されていない場合は実行できません．google drive 上に "data.zip" (**831MB**) を配置することが可能であれば，"data_download.ipynb" を先に実行してください．難しい場合は，omnicampus 演習環境を利用してください．．



In [ ]:
# omnicampus 上では 4 セル目まで実行不要
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# データダウンロード用の notebook にてgoogle drive への保存後，
# 反映に時間がかかる可能性がありますので，google drive のマウント後，
# data.zip がディレクトリ内にあることを確認してから実行してください．
# data.zip を /content 下にコピーする
!cp "/content/drive/MyDrive/DLBasics2025_colab/最終課題/NYUv2/data.zip" "/content"

In [ ]:
# カレントディレクトリ下のファイル群を確認
# data.zip が表示されれば問題ないです
%ls

data/  data.zip  drive/  sample_data/


In [ ]:
# データを解凍する
!unzip data.zip
!mkdir data
!mv train test data/

Archive:  data.zip
replace data/test/depth/000833.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: mkdir: cannot create directory ‘data’: File exists
mv: cannot stat 'train': No such file or directory
mv: cannot stat 'test': No such file or directory


omnicampus 演習環境では，data_download.ipynb のマウント，zip 化，drive へのコピーを実行しないことで，"data.zip" を解凍した形で配置されます．したがって，data ディレクトリが存在するディレクトリをカレントディレクトリとするだけで良いです．



In [ ]:
# omnicampus 実行用
# 以下の例では/workspace/Segmentation/split_data_scripts/omnicampus に data ディレクトリがあると想定
# %cd /workspace/Segmentation/split_data_scripts_omnicampus

In [ ]:
# !pip install numpy==1.22.2 h5py scikit-image

# import library

In [ ]:
!pip install torch torchvision timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
import os
import time
from tqdm import tqdm
import numpy as np
from scipy.io import loadmat
from PIL import Image
import torch
import torch.nn as nn
from torch import optim
import torch.utils.data as data
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import VisionDataset
from torchvision import transforms
from torchvision.transforms import (
    Compose,
    RandomResizedCrop,
    RandomHorizontalFlip,
    ColorJitter,
    GaussianBlur,
    Resize,
    ToTensor,
    Normalize,
    Lambda,
    InterpolationMode
)
from torch.cuda.amp import autocast, GradScaler
from dataclasses import dataclass
import random
import timm

# DataLoader

In [ ]:
# カラーマップ生成関数：セグメンテーションの可視化用
def colormap(N=256, normalized=False):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap/255 if normalized else cmap
    return cmap

# NYUv2データセット：RGB画像、セgmentation、深度、法線マップを提供するデータセット
class NYUv2(VisionDataset):
    """NYUv2 dataset

    Args:
        root (string): Root directory path.
        split (string, optional): 'train' for training set, and 'test' for test set. Default: 'train'.
        target_type (string, optional): Type of target to use, ``semantic``, ``depth``.
        transform (callable, optional): A function/transform that takes in a PIL image and returns a transformed version.
        target_transform (callable, optional): A function/transform that takes in the target and transforms it.
        depth_transform (callable, optional): A function/transform that takes in a PIL depth image and returns a transformed version.
    """
    cmap = colormap()
    def __init__(self,
                 root,
                 split='train',
                 include_depth=False,
                 transform=None,
                 target_transform=None,
                 depth_transform=None
                 ):
        super(NYUv2, self).__init__(root, transform=transform, target_transform=target_transform)

        # データセットの基本設定
        assert(split in ('train', 'test'))
        self.root = root
        self.split = split
        self.include_depth = include_depth
        self.train_idx = np.array([255, ] + list(range(13)))  # 13クラス分類用
        self.depth_transform = depth_transform

        # 画像ファイルのパスリストを作成
        img_names = os.listdir(os.path.join(self.root, self.split, 'image'))
        img_names.sort()
        images_dir = os.path.join(self.root, self.split, 'image')
        self.images = [os.path.join(images_dir, name) for name in img_names]

        label_dir = os.path.join(self.root, self.split, 'label')
        if (self.split == 'train'):
          self.labels = [os.path.join(label_dir, name) for name in img_names]
          self.targets = self.labels

        depth_dir = os.path.join(self.root, self.split, 'depth')
        self.depths = [os.path.join(depth_dir, name) for name in img_names]

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB') # Ensure RGB
        depth = Image.open(self.depths[idx]).convert('L') # Ensure grayscale

        if self.transform is not None:
            image = self.transform(image)
        if self.depth_transform is not None:
            depth = self.depth_transform(depth)

        if self.include_depth:
            # Stack RGB and Depth channels
            image = torch.cat([image, depth], dim=0)

        if self.split=='test':
            return image

        if self.split == 'train' and self.target_transform is not None:
            target = Image.open(self.targets[idx])
            target = self.target_transform(target)

        return image, target

    def __len__(self):
        return len(self.images)

# Model Section


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else cpu)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0,5, 0.5), std=(0.5, 0.5, 0.5))
])

num_classes = 13

model = timm.create_model("vit_base_patch16_224", pretrained=True)
model.head = nn.Linear(model.head.in_features, num_classes)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
# 2つの畳み込み層とバッチ正規化、ReLUを含むブロック
# UNetの各層で使用される基本的な畳み込みブロック
# class DoubleConv(nn.Module):
#    def __init__(self, in_channels, out_channels):
#         super().__init__()
#         self.double_conv = nn.Sequential(
#             nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
#             nn.BatchNorm2d(out_channels),
#             nn.ReLU(inplace=True),
#         )

#     def forward(self, x):
#         return self.double_conv(x)

# UNetモデル：エンコーダ・デコーダ構造のセグメンテーションモデル
# class UNet(nn.Module):
#     def __init__(self, in_channels, num_classes):
#         super().__init__()
#         # エンコーダ部分：特徴量の抽出と空間サイズの縮小
#         self.enc1 = DoubleConv(in_channels, 64)
#         self.enc2 = DoubleConv(64, 128)
#         self.enc3 = DoubleConv(128, 256)
#         self.enc4 = DoubleConv(256, 512)
#         self.pool = nn.MaxPool2d(2)

#         # デコーダ部分：特徴量の統合と空間サイズの復元
#         self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
#         self.dec3 = DoubleConv(512 + 256, 256)
#         self.dec2 = DoubleConv(256 + 128, 128)
#         self.dec1 = DoubleConv(128 + 64, 64)

#         # 最終層：クラス数に応じた出力チャネルに変換
#         self.final = nn.Conv2d(64, num_classes, kernel_size=1)

#     def forward(self, x):
#         # エンコーダパス：特徴抽出とダウンサンプリング
#         e1 = self.enc1(x)
#         e2 = self.enc2(self.pool(e1))
#         e3 = self.enc3(self.pool(e2))
#         e4 = self.enc4(self.pool(e3))

#         # デコーダパス：特徴統合とアップサンプリング（スキップ接続を使用）
#         d3 = self.dec3(torch.cat([self.up(e4), e3], dim=1))
#         d2 = self.dec2(torch.cat([self.up(d3), e2], dim=1))
#         d1 = self.dec1(torch.cat([self.up(d2), e1], dim=1))

#         return self.final(d1)


# Train and Valid

In [ ]:
# config
@dataclass
class TrainingConfig:
    # データセットパス
    dataset_root: str = "data"

    # データ関連
    batch_size: int = 32
    num_workers: int = 4

    # モデル関連
    in_channels: int = 3
    num_classes: int = 13  # NYUv2データセットの場合

    # 学習関連
    epochs: int = 100
    learning_rate: float = 0.001
    weight_decay: float = 1e-4

    # データ分割関連
    train_val_split: float = 0.8  # 訓練データの割合

    # デバイス設定
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    # チェックポイント関連
    checkpoint_dir: str = "checkpoints"
    save_interval: int = 5  # エポックごとのモデル保存間隔

    # データ拡張・前処理関連
    image_size: tuple = (256, 256)
    normalize_mean: tuple = (0.485, 0.456, 0.406)  # ImageNetの標準化パラメータ
    normalize_std: tuple = (0.229, 0.224, 0.225)

    def __post_init__(self):
        import os
        os.makedirs(self.checkpoint_dir, exist_ok=True)

In [ ]:
def set_seed(seed):
    """
    シードを固定する．

    Parameters
    ----------
    seed : int
        乱数生成に用いるシード値．
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
set_seed(42)
# 設定の初期化
config = TrainingConfig(
    dataset_root='/content/data',
    batch_size=16,
    num_workers=4,
    learning_rate=1e-4,
    epochs=100,
    image_size=(320, 240),
    in_channels=4  # RGB(3チャネル) + Depth(1チャネル)
)

'''
データセットのディレクトリ構造：
    data/NYUv2/
    ├─train/
    │  ├─image/      # RGB画像（入力）
    │  │    000000.png
    │  │    ...
    |  ├─depth/      # 深度画像（入力）
    |  │    000000.png
    |  │    ...
    │  └─label/      # 13クラスセグメンテーション（教師ラベル）
    │       000000.png
    │       ...
    └─test/
       ├─image/      # RGB画像（入力）
       │    000000.png
       │    ...
       ├─depth/      # 深度画像（入力）
       │    000000.png
       │    ...
'''


# ------------------
#    Dataloader
# ------------------

# データ前処理の定義
# RGB画像のTransform：リサイズとテンソル変換、正規化
transform = Compose([
    Resize(config.image_size, interpolation=InterpolationMode.BILINEAR),
    ToTensor(),
    Normalize(mean=config.normalize_mean, std=config.normalize_std) # Use ImageNet stats for RGB
])

# Depth画像のTransform：リサイズとテンソル変換、正規化 (単一チャンネル)
depth_transform = Compose([
    Resize(config.image_size, interpolation=InterpolationMode.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5,), std=(0.5,)) # Normalize depth to [-1, 1]
])


# セグメンテーションラベルのTransform：リサイズとテンソル変換
target_transform = Compose([
    Resize(config.image_size, interpolation=InterpolationMode.NEAREST),
    Lambda(lambda lbl: torch.from_numpy(np.array(lbl)).long())
])

# データセットの準備
# RGBデータセットとセグメンテーションラベルの読み込み
train_dataset = NYUv2(
    root=config.dataset_root,
    split='train',
    include_depth=True,
    transform=transform,
    target_transform=target_transform,
    depth_transform=depth_transform
)

# テストデータセット
test_dataset = NYUv2(
    root=config.dataset_root,
    split='test',
    include_depth=True,
    transform=transform,
    depth_transform=depth_transform
)


'''
    train data:
        Type of batch: tuple
        Index 0 (入力データ):
            Type: torch.Tensor
            Shape: torch.Size([Batch, 4, N, M])
            Details: RGBDテンソル (RGB + Depth)
                    - チャネル0-2: RGB画像 (正規化済み)
                    - チャネル3: 深度画像 (正規化済み)
        Index 1 (教師ラベル):
            Type: torch.Tensor
            Shape: torch.Size([Batch, N, M])
            Details: セグメンテーションマップ
                    - 値域: 0-12 (13クラス)
                    - 255: ignore index

    test data:
        Type of batch: torch.Tensor
        Shape: torch.Size([Batch, 4, N, M])
        Details: RGBD画像 (正規化済み)
'''

# データローダーの作成
train_data = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
test_data = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=config.num_workers)

# モデルとトレーニングの設定
device = config.device
print(f"Using device: {device}")

# ------------------
#    Model
# ------------------
# model = UNet(in_channels=config.in_channels, num_classes=config.num_classes).to(device)

# ------------------
#    optimizer
# ------------------
optimizer = optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
criterion = nn.CrossEntropyLoss(ignore_index=255)

# ------------------
#    Training
# ------------------
num_epochs = config.epochs
scaler = GradScaler()

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    print(f"on epoch: {epoch+1}")
    for inputs, labels in train_data: # inputs will be the concatenated RGBD tensor
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_data)}')

# モデルの保存
current_time = time.strftime("%Y%m%d%H%M%S")
model_path = f"model_{current_time}.pt"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Using device: cuda
on epoch: 1


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-22-3898750863.py:116: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipython-input-16-3380550403.py", line 69, in __getitem__
    image = self.transform(image)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torchvision/transforms/transforms.py", line 95, in __call__
  

In [ ]:
# ------------------
#    Evaluation
# ------------------

model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# 予測結果の生成
predictions = []

with torch.no_grad():
    print("Generating predictions...")
    for inputs in test_data: # inputs will be the concatenated RGBD tensor
        inputs = inputs.to(device)
        output = model(inputs)            # [Batch, num_classes, H, W]
        pred = output.argmax(dim=1)  # [Batch, H, W]
        predictions.append(pred.cpu())
predictions = torch.cat(predictions, dim=0)

predictions = predictions.cpu().numpy()
np.save('submission.npy', predictions)
print("Predictions saved to submission.npy")

Generating predictions...


100%|██████████| 654/654 [00:23<00:00, 27.80it/s]


Predictions saved to submission.npy


## 提出方法

以下の3点をzip化し，Omnicampusの「最終課題 (セグメンテーション)」から提出してください．

- `submission.npy`
- `model.pt`や`model_best.pt`など，テストに使用した重み（拡張子は`.pt`のみ）
- 本Colab Notebook

In [ ]:
from zipfile import ZipFile, ZIP_DEFLATED

notebook_path = "/content/drive/MyDrive/DLBasics2025_colab/最終課題/NYUv2"

with ZipFile("submission.zip",
             mode="w",
             compression=ZIP_DEFLATED,
             compresslevel=9) as zf:
    zf.write("submission.npy")
    zf.write(model_path)
    zf.write(notebook_path,
             arcname="DL_Basic_2025_Competition_NYUv2_baseline.ipynb")